In [1]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, CSVLogger


Using TensorFlow backend.


In [2]:
#Load Data

train = pd.read_csv("C:/Users/haima/Desktop/Python_working/digit-recognizer/train.csv")
test = pd.read_csv("C:/Users/haima/Desktop/Python_working/digit-recognizer/test.csv")

y = train["label"]
x = train.drop(labels = ["label"], axis = 1) 

In [3]:
# Reshape data

x = x.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)


In [4]:
# One hot encoding

y = to_categorical(y, num_classes=10)

In [5]:
#Training and validation sets

x_train, x_val, y_train, y_val = train_test_split(x, y,test_size=0.1,shuffle = True)

In [6]:
# Model

model = Sequential()

model.add(Conv2D(filters=32,
                 kernel_size=(5,5),
                 padding='Same', 
                 activation='relu',
                 input_shape=(28,28,1)))

model.add(Conv2D(filters=32,
                 kernel_size=(5,5),
                 padding='Same', 
                 activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(0.5))



model.add(Conv2D(filters=64, 
                 kernel_size=(3,3),
                 padding='Same', 
                 activation='relu'))

model.add(Conv2D(filters=64,
                 kernel_size=(3,3),
                 padding='Same', 
                 activation='relu'))

model.add(MaxPool2D(pool_size=(2,2),
                    strides=(2,2)))

model.add(Dropout(0.5))



model.add(Flatten())

model.add(Dense(8192, activation='relu'))

model.add(Dropout(0.5))



model.add(Dense(2048, activation='relu'))

model.add(Dropout(0.5))



model.add(Dense(10, activation="softmax"))


model.compile(optimizer= RMSprop(lr=0.0001,
                                rho=0.9,
                                epsilon=1e-08,
                                decay=0.00001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [7]:
# Data augmentation

data_gen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=1,
                                    zoom_range=0.1, 
                                    width_shift_range=0.05,
                                    height_shift_range=0.05)
data_gen.fit(x_train)


In [8]:
# Training the model

history = model.fit_generator(data_gen.flow(x_train,
                                                  y_train,
                                                  batch_size=100),
                              epochs=30,
                              validation_data=(x_val, y_val),
                              verbose=1,
                              steps_per_epoch=x_train.shape[0] // 256)

model.save_weights("model.h5")

Epoch 1/30
147/147 [==============================] - 181s 1s/step - loss: 1.1352 - accuracy: 0.6176 - val_loss: 44.2691 - val_accuracy: 0.8900
Epoch 2/30
147/147 [==============================] - 165s 1s/step - loss: 0.4702 - accuracy: 0.8527 - val_loss: 25.9768 - val_accuracy: 0.9414
Epoch 3/30
147/147 [==============================] - 162s 1s/step - loss: 0.2838 - accuracy: 0.9093 - val_loss: 16.7699 - val_accuracy: 0.9667
Epoch 4/30
147/147 [==============================] - 160s 1s/step - loss: 0.2262 - accuracy: 0.9314 - val_loss: 12.9645 - val_accuracy: 0.9707
Epoch 5/30
147/147 [==============================] - 161s 1s/step - loss: 0.1793 - accuracy: 0.9450 - val_loss: 11.9563 - val_accuracy: 0.9745
Epoch 6/30
147/147 [==============================] - 160s 1s/step - loss: 0.1587 - accuracy: 0.9508 - val_loss: 10.0317 - val_accuracy: 0.9805
Epoch 7/30
147/147 [==============================] - 160s 1s/step - loss: 0.1407 - accuracy: 0.9546 - val_loss: 9.8373 - val_accuracy: 

In [9]:
# Testing and predictions

predictions = model.predict_classes(test, verbose=1)
pd.DataFrame({"ImageId":list(range(1,len(predictions)+1)),
              "Label":predictions}).to_csv("C:/Users/haima/Desktop/Python_working/digit-recognizer/submission.csv",
                                           index=False,
                                           header=True)

28000/28000 [==============================] - 169s 6ms/step
